## HarmonyPy Introduction

This notebook demonstrates several basic examples highlighting how to query and access customized data outputs from NASA Earthdata Harmony. See https://harmony-py.readthedocs.io/en/latest/ for detailed documentation on HarmonyPy.

### Import packages

First we import packages needed to request and visualize our data, as well as the `harmony-py` library itself. Make sure to install `harmony-py` and its dependencies into your current Python environment prior to executing the notebook:

```
pip install -U harmony-py
```

In [ ]:
# Install notebook requirements
import sys
import helper
# Install the project and 'examples' dependencies
helper.install_project_and_dependencies('..', libs=['examples'])

In [ ]:
import datetime as dt
from IPython.display import display, JSON
import os
import rasterio
import rasterio.plot

from harmony import BBox, WKT, Client, Collection, Request
from harmony.config import Environment

### Quick start

You can request and download data using `harmony-py` in just a few easy lines. Although more advanced subsetting and transformation options may be supported on your data product of interest, this example below demonstrates a basic spatial bounding box and temporal range request, using the direct download method: 

```
harmony_client = Client(auth=('EDL_username', 'EDL_password'))
request = Request(
    collection=Collection(id=dataset_short_name),
    spatial=BBox(w, s, e, n),
    temporal={
        'start': dt.datetime(yyyy, mm, dd),
        'stop': dt.datetime(yyyy, mm, dd)
    }
)
job_id = harmony_client.submit(request)
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
```

The guidance below offers more detailed examples highlighting many of the helpful features provided by the Harmony Py library, including direct s3 access from Earthdata Cloud-hosted data if running in the AWS us-west-2 region.

### Create Harmony Client object

First, you will need to create your Harmony Client, which is what you will interact with to submit and inspect a data request to Harmony, as well as to retrieve your results. 

When creating the Client, you need to provide either your EDL token or your [Earthdata Login](https://urs.earthdata.nasa.gov) credentials, which are required to access data from NASA EOSDIS. There are four options for providing your Earthdata Login token or username and password when creating a Harmony Client: 

1. Provide EDL token using environment variable:

```
$ export EDL_TOKEN='my_eld_token'
```

2. Provide your username and password directly when creating the client:

```
harmony_client = Client(auth=('captainmarvel', 'marve10u5'))
```

3. Set your credentials using environment variables:

You can either export these directly: 

```
$ export EDL_USERNAME='captainmarvel'
$ export EDL_PASSWORD='marve10u5'
```
Or by storing them in a .env file, which operates in a similar fashion to .netrc. You will need to store the file in your current working directory and it must be named .env with the following format:

```
EDL_USERNAME=myusername
EDL_PASSWORD=mypass
```

4. Use a .netrc file:

Create a .netrc file in your home directory, using the example below:

```
machine urs.earthdata.nasa.gov
login captainmarvel
password marve10u5
```

The code below will create a Harmony Client using EDL token if the `EDL_TOKEN` envrionment variable is defined or use .netrc file if the `EDL_TOKEN` envrionment variable is not defined.

In [ ]:
token = os.getenv('EDL_TOKEN')
if (token is None or token == ''):
    print('Use .netrc for client authentication')
    harmony_client = Client()
else:
    print('Use EDL token from the environment for client authentication')
    harmony_client = Client(token=token)

### Create Harmony Request

The following are common request parameters:

* `collection`: Required parameter. This is the NASA EOSDIS collection, or data product. There are two options for inputting a collection of interest:
    1. Provide a concept ID, which is an ID provided in the Common Metadata Repository (CMR) metadata
    2. Data product short name (e.g. SENTINEL-1_INTERFEROGRAMS). 
* `spatial`: Bounding box or Well Known Text (WKT) spatial constraints on the data. The Harmony `Bbox` class accepts spatial coordinates as decimal degrees in w, s, e, n order, where longitude = -180, 180 and latitude = -90, 90. The Harmony `WKT` class accepts WKT string as spatial constraints.
* `shape`: Perform a shapefile subsetting request on a supported collection by passing the path to a GeoJSON file (*.json or .geojson), an ESRI Shapefile (.zip or .shz), or a kml file (.kml) as the "shape" parameter
* `temporal`: Date/time constraints on the data. The example below demonstrates temporal start and end ranges using the python `datetime` library.

Other advanced parameters that may be of interest. Note that many reformatting or advanced projection options may not be available for your requested dataset. See the [documentation](https://harmony-py.readthedocs.io/en/latest/) for details on how to construct these parameters.

* `crs`: Reproject the output coverage to the given CRS. Recognizes CRS types that can be inferred by gdal, including EPSG codes, Proj4 strings, and OGC URLs (http://www.opengis.net/def/crs/…)
* `interpolation`: specify the interpolation method used during reprojection and scaling
* `scale_extent`: scale the resulting coverage either among one axis to a given extent
* `scale_size`: scale the resulting coverage either among one axis to a given size
* `granule_id`: The CMR Granule ID for the granule (file) which should be retrieved
* `width`: number of columns to return in the output coverage
* `height`: number of rows to return in the output coverage
* `format`: the output mime type to return
* `max_results`: limits the number of input files processed in the request

The example utilized in this tutorial demonstrates a shapefile subset of the Big Island of Hawaii on February 24, 2020. A bounding box subset over the Mauna Kea and Mauna Loa volcanoes is also commented out below to show a similar subsetting option. The SENTINEL-1_INTERFEROGRAMS dataset, distributed by the ASF DAAC, is a prototype Level 2 NISAR-Format product. See https://asf.alaska.edu/data-sets/derived-data-sets/sentinel-1-interferograms/ for more information. 

This request specifies a subset of the unwrappedPhase variable, in TIFF format, with a maximum file result capped at 2 for demonstration purposes. 

#### ___Note that a Sentinel-3 End-User License Agreement (EULA) is required to access these data.___
#### ___Please go to https://grfn.asf.alaska.edu/door/download/S1-GUNW-D-R-021-tops-20201029_20191029-033636-28753N_27426N-PP-2dde-v2_0_3.nc  to initiate a file download, which will first prompt you to accept the required EULA if you have not already done so. If you do not accept this EULA, you will receive an error when submitting your Harmony request.___

In [ ]:
shapefile_path = 'Big_Island_0005.zip'

request = Request(
    collection=Collection(id='SENTINEL-1_INTERFEROGRAMS'),
    # spatial=BBox(-155.75, 19.26, -155.3, 19.94), # bounding box example that can be used as an alternative to shapefile input
    # spatial=WKT('POLYGON((-155.75 19.26, -155.3 19.26, -155.3 19.94, -155.75 19.94, -155.75 19.26))'), # WKT example
    shape=shapefile_path,
    temporal={
        'start': dt.datetime(2020, 2, 24),
        'stop': dt.datetime(2020, 2, 25),
    },
    variables=['science/grids/data/unwrappedPhase'],
    format='image/tiff',
    max_results=2,
    # If desired, deliver results to a custom destination bucket. Note the bucket must reside in AWS us-west-2 region.
    # destination_url='s3://my-bucket'
)

### Check Request validity

Before submitting a Harmony Request, you can test your request to see if it's valid and how to fix it if not. In particular, `request.is_valid` will check to ensure that your spatial BBox bounds and temporal ranges are entered correctly.  

In [ ]:
print(f"Request valid? {request.is_valid()}")
for m in request.error_messages():
    print(" * " + m)

### Submit Request

Once you have created your request, you can now submit it to Harmony using the Harmony Client object. A job id is returned, which is a unique identifier that represents your submitted request.

By default the job will have a 'harmony_py' label. This can be disabled by setting the `EXCLUDE_DEFAULT_LABEL` environment variable to "true" before making the request.

In [ ]:
job_id = harmony_client.submit(request)
job_id

### Check Request status

You can check on the progress of a processing job with `status()`. This method blocks while communicating with the server but returns quickly. The JSON helper function provides a nicer formatting option for viewing the status message. 

In [ ]:
JSON(harmony_client.status(job_id))

Depending on the size of the request, you may want to wait until the request has completed processing before the remainder of your code is executed. The `wait_for_processing()` method will block subsequent lines of code while optionally showing a progress bar.

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

### View Harmony job response and output URLs

Once your data request has finished processing, you can view details on the job that was submitted to Harmony, including the API call to Harmony, and informational messages on your request if available. 

`result_json()` calls `wait_for_processing()` and returns the complete job in JSON format once processing is complete. You may optionally show the progress bar using the same `show_progress=True` statement in the previous line. 

In [ ]:
data = harmony_client.result_json(job_id)
JSON(data)

### Download and visualize Harmony output files

The files returned from your request are returned as HTTPS URLs in all cases other than when requesting Zarr as an output format(s3 URLs are returned instead to support accessing and interacting with those outputs in the cloud). For all Harmony requests, direct s3 URLs are also provided within a STAC catalog provided in the job response as we'll explore in more detail below. 

#### Retrieve list of output URLs

The `result_urls()` method calls `wait_for_processing()` and returns a list of the processed data URLs once processing is complete. You may optionally show the progress bar as shown below.

In [ ]:
urls = harmony_client.result_urls(job_id, show_progress=True)
urls

#### Download files

The next code block utilizes `download_all()` to download all data output file URLs. This is a non-blocking step during the download itself, but this line will block subsequent code while waiting for the job to finish processing. You can optionally specify a directory and specify whether to overwrite existing files as shown below.

You may call `result()` on future objects (those that are awaiting processing) to realize them. A call to `result()` blocks until that particular future object finishes downloading. Other future objects will download in the background, in parallel. When downloading is complete, the future objects will return the file path string of the file that was just downloaded. This file path can then be fed into other libraries that may read the data files and perform other operations.

In [ ]:
results = harmony_client.download_all(job_id, directory='/tmp', overwrite=True)
file_names = [f.result() for f in results]
file_names

With `download()`, this will download only the URL specified, in case you would like more control over individual files.

In [ ]:
file_name = harmony_client.download(next(urls), overwrite=True).result()
file_name

### Visualize Downloaded Outputs

The output image files can be visualized using the `Rasterio` library.

In [ ]:
for f in file_names:
    rasterio.plot.show(rasterio.open(f))

### Explore output STAC catalog and retrieve results from s3

A [STAC](https://stacspec.org/) catalog is returned in each Harmony request result. The stac items include not only the s3 locations for each output file, but also the spatial and temporal metadata representing each subsetted output. `stac_catalog_url` will return the URL of the catalog, and can also accept an optional progress status if desired.

In [ ]:
stac_catalog_url = harmony_client.stac_catalog_url(job_id)
stac_catalog_url

#### Using PySTAC:

Following the directions for PySTAC (https://pystac.readthedocs.io/en/latest/quickstart.html), we can view the timestamp and s3 locations of each STAC item:

In [ ]:
from pystac import Catalog

cat = Catalog.from_file(stac_catalog_url)

print(cat.title)
s3_links = []
for item in cat.get_all_items():
    print(item.datetime, [asset.href for asset in item.assets.values()])
    s3_links.append([asset.href for asset in item.assets.values()])

#### Using intake-stac:

View each item value returned:

In [ ]:
import intake
cat = intake.open_stac_catalog(stac_catalog_url)
display(list(cat))

And the metadata contents of each item:

In [ ]:
entries = []
for id, entry in cat.search('type').items():
    display(entry)
    entries.append(entry)

### Cloud in-place access 

**Note that the remainder of this tutorial will only succeed when running this notebook within the AWS us-west-2 region.** 

Harmony data outputs can be accessed within the cloud using the s3 URLs and AWS credentials provided in the Harmony job response. Below are examples using both `intake-stac` or `boto3` to access the data in the cloud. 

#### AWS credential retrieval

Using `aws_credentials` you can retrieve the credentials needed to access the Harmony s3 staging bucket and its contents.

In [ ]:
# NOTE: if you specified destination_url you'll have to retrieve your credentials in another manner
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html
creds = harmony_client.aws_credentials()
creds

#### Using boto3

In [ ]:
#
# NOTE: Execution of this cell will only succeed within the AWS us-west-2 region.
#

import boto3

s3 = boto3.client('s3', **creds)
for i in range(len(s3_links)):
    uri = s3_links[i][0]
    bucket = uri.split('/')[2]
    obj = '/'.join(uri.split('/')[3:])
    fn = uri.split('/')[-1]
    with open(fn, 'wb') as f:
        s3.download_fileobj(bucket, obj, f)

#### Using intake-stac

Once again, you can use `intake-stac` to directly access each output from Harmony in AWS. Viewing the file structure and plotting the image can be done in a few simple lines when working with the data in-region:

In [ ]:
#
# NOTE: Execution of this cell will only succeed within the AWS us-west-2 region.
#

for i in range(len(list(cat))):
    da = cat[list(cat)[i]][entries[i].describe()['name']].to_dask()
    display(da)
    da.plot()